In [104]:
import os,math,sys, numpy as np
import csv
import networkx as nx
np.set_printoptions(threshold=np.nan)
import matplotlib
from collections import defaultdict
import glob
from sklearn.decomposition import NMF
from sklearn.cluster import KMeans
from NMF_python_sparse import nmf
import glob
from datetime import date
import time
import pandas as pd

In [105]:
##############
##This is the main subroutine that calls all of the above scripts. 
#Need to specify the type of data, e.g hMor or hDor or hMor_Beta etc.
##Input data is a tab separated .txt NxM where N is the number of compounds and M is the number of assays
##Labels for N and M should be in the .txt file

#Review_hDor
filetype = input('What is the datatype name? e.g. Review_hDor')

##To convert this to a .py use the line below instead of the line above
#filetype='{0}'.format(sys.argv[1])

What is the datatype name? Review_hDor


In [117]:
##In order to test statistical robustness, at some point we need to create randomized data from the original data
#and then get randomized clustering.
##Setting this flag =='Yes' will repeat the clustering process while randomly shuffling the data
random_flag=input('Is this for random simulations? (Yes, No) ')
#random_flag='{0}'.format(sys.argv[2])

Is this for random simulations? (Yes, No) Yes


In [106]:
try:#Creating directories if this is the first time this data type has been run. Files are relative to code location
    os.mkdir('../SEM/%s/'%(filetype))
except:
    ''
try:
    os.mkdir('../SEM/%s/FrequencyMatrix/'%(filetype))#=
    os.mkdir('../SEM/%s/Standardizedmatrices/'%(filetype))
    os.mkdir('../SEM/%s/Randommatrices/'%(filetype))
except:
    ''
today=date.fromtimestamp(time.time())
today='Jan-21-2019'
print(today)

Jan-21-2019


In [115]:
##Read in data
##Here I have manually set the expected inputs for each file. These would need to be adjusted for different inputs
##I wanted to control what the columns were called, with a specific syntax, that is why I hard coded it here

assay_list=['barr1','barr2','barr2_grk2','barr2_grk5','barr2_grk6','camp','gai1','gai2','gao','kir']
measure_list=['pEC50','pEC50_SEM','Emax','Emax_SEM','LogT','LogT_SEM','LogKa','LogKa_SEM','LogR','LogR_SEM','n']

tuples=[(i,j) for i in assay_list for j in measure_list]

data_df=pd.read_csv('../SEM/Data/%s/Data_%s.txt'%(filetype,filetype),sep='\t',index_col=0, skiprows=[0,1],
                    header=None)
data_df.columns=pd.MultiIndex.from_tuples(tuples )

##NR and NF are being considered as 0's currently
data_df.replace('NR',0, inplace=True)
data_df.replace('NF',0,inplace=True)


In [116]:
##Define which assays and measures we care about for the clustering
assays_of_int=['barr1','barr2','barr2_grk2','barr2_grk5','barr2_grk6','camp','gai1','gai2','gao','kir']
measures_of_int=['Emax','Emax_SEM','LogT','LogT_SEM','LogR','LogR_SEM']
measure_mean_list=['Emax','LogT','LogR']
measure_sem_list=['Emax_SEM','LogT_SEM','LogR_SEM']

##list of compounds to cluster
mutationlist=list(data_df.index.values)


##dataframe of replicates for each assay
replicate_df=data_df.loc[:,(assays_of_int,'n')]

##Dataframe of actual mean and SEM values


select_data_df=data_df.loc[:,(assays_of_int,measures_of_int)]
mean_df=data_df.loc[:,(assays_of_int,measure_mean_list)]
sem_df=data_df.loc[:,(assays_of_int,measure_sem_list)]


In [109]:
def sample_from_SEM(col):
    #Initial data is a combination of mean values and SEM from a dose response curve, in order to include the 
    #variation around each mean we are sampling from the distribution for each point for each compound
    random_array=[]
    index_name=col.name
    new_index_name=(index_name[0],'%s_SEM'%(index_name[1]))
    
    for compound in col.index.values:
        mean_value=col[compound]
        sem_value=sem_df.loc[:,new_index_name][compound]
        num_replicates=replicate_df.loc[compound,(index_name,'n')][0]
        if pd.isnull(mean_value)==True or sem_value==0:
            randomvalue=mean_value
        else:
            stdev=float(sem_value)*float(num_replicates)**0.5#converting SEM to stdev using number of replicates
            randomvalue=np.random.normal(float(mean_value),stdev,1)#select a single value from the distribution around each mean value
            randomvalue=randomvalue[0]
            
        random_array.append(randomvalue)
    error_df[col.name]=random_array

In [110]:
def standardizemat(erroredmat):
    #In order to prevent any bias induced by scale differences between measures, 
    #we standardize each column between 0-1 based on the max and min of that column
    normtrans=np.transpose(erroredmat)
    standmat=np.zeros(normtrans.shape)
    for rowcounter,i in enumerate(normtrans):
        maxvalue=np.nanmax(i)
        minvalue=np.nanmin(i)
        colcounter=0
        for colcounter,j in enumerate(i):
            if j!='NaN':
                tempvalue=(float(j)-minvalue)/float(maxvalue-minvalue)
                standmat[rowcounter][colcounter]=tempvalue
            else:
                standmat[rowcounter][colcounter]='NaN'
    standmat=np.transpose(standmat)
    return(standmat)#output is a standardized matrix of the same dimensions as the sampled "erroredmat"


In [111]:
def Freqaverage(repl,mutationlist,filetype,today):
    #Method to average the frequency matrix over all K's and format the output with labels.
    rep=repl
    nmfmatrix=np.zeros(shape=(len(mutationlist),len(mutationlist)))#initialize matrix
    filecounter=0
    path='../SEM/%s/FrequencyMatrix/'%(filetype)
    for filename in glob.glob(os.path.join(path,'Frequencymatrix_%s_rep=%s*'%(today,rep))):#loop over all frequency matrices within this replicat
        file=open(filename)
        linecounter=0
        tempmatrix=np.loadtxt(filename)
        nmfmatrix=nmfmatrix+tempmatrix
        filecounter=filecounter+1
    
    out_df=pd.DataFrame(nmfmatrix, index=mutationlist,columns=mutationlist)
    outfile_name='../SEM/%s/FrequencyMatrix/FrequencyMatrix_rep=%s_Average.txt'%(filetype,rep)
    out_df.to_csv(outfile_name, header=True, index=False, sep='\t', mode='a')
    nmfmatrix=nmfmatrix/float(filecounter)
    
    return nmfmatrix

In [135]:
def randomizeclusters(erroredmat):#takes the sampled data matrix and randomizes all the values. returns a shuffled matrix as output
    matshape=np.shape(erroredmat)
    row=matshape[0]
    col=matshape[1]
    
    rowarray=list(range(0,row))
    colarray=list(range(0,col))
    
    np.random.shuffle(rowarray)
    np.random.shuffle(colarray)
    
    indexlist=[]
    for i in rowarray:
        for k in colarray:
            index=str(i)+'-'+str(k)
            indexlist.append(index)
    np.random.shuffle(indexlist)

    Xrand=np.zeros((row,col))
    rowcounter=0
    indexcounter=0
    for i in Xrand:
        colcounter=0
        for k in i:
            #print(rowcounter,colcounter,indexcounter,index)
            index=indexlist[indexcounter].split('-')

            temprow=int(index[0])
            tempcol=int(index[1])
            Xrand[rowcounter][colcounter]=erroredmat[temprow][tempcol]
            indexcounter+=1

            colcounter+=1
            ''
        rowcounter+=1
    return Xrand

In [139]:
replicates=10#This is where you define the number of total replicates. This will include new sampling from the SEM values within each replicate. 
numit=10#Within each replicate you need to define the number of iterations to run the NMF/Kmeans method at each K

totalfreqmatrix=np.zeros((len(mutationlist),len(mutationlist)))#initializing the final frequency matrix
if random_flag=='No':
    for repl in range(0,replicates):
        print(repl)
        ##randomly sampling mean and std in order to create new matrix
        error_df= pd.DataFrame(index=data_df.index.values, columns=data_df.loc[:,(assays_of_int,measure_mean_list)].columns)
        error_df.fillna(0,inplace=True)
        mean_df.apply(sample_from_SEM, axis=0)

        erroredmat_df.to_csv('../SEM/%s/Randommatrices/randommatrix_%s_%s.csv'%(filetype,today,str(repl)))
        erroredmat=error_df.values

        #standize the matrix values using the max and min within each assay column (value-min)/(max-min) 
        #where max and min are column specific
        standmat=standardizemat(erroredmat)
        np.savetxt('../SEM/%s/Standardizedmatrices/Standardmatrix_%s_%s.txt'%(filetype,today,str(repl)),standmat)

        freqmatrix=np.zeros((len(error_df),len(error_df)))

        ##############################################
        ###Begin clustering algorithm using the standardized sampled matrix you just created above
        nmfmatrix=np.zeros(shape=(len(error_df),len(error_df)))
        for basis in range(2,8):#Will perform nmf/kmeans method over each K between this range
            for clustk in range(2,8):
                if basis==clustk:#Only running if K=K for nnmf and Kmeans. This could be removed if you want all iterations 2,2; 2,3; 2,4;....8,7; 8,8
                    freqmatrix=np.zeros((len(error_df),len(error_df)))
                    for i in range(0,numit):
                        print('This is replicate',repl,'and iteration=',i,' with basis and clustk=',basis)

                        W,H=nmf(standmat,clustk)#Creating basis vectors. ONly W is used in this script, but H is used in the Assay clustering script
                        clusterdata=KMeans(n_clusters=clustk).fit_predict(W)

                        for j in range (0,len(clusterdata)):
                            #Quantifying if two compounds occur in the same cluster or not. Appending a an overall frequency matrix specific to this replicate.
                            #So for each replicate, a frequency matrix created which calculates how often all compounds cluster together over the number of numit for Each K
                            for l in range (0,len(clusterdata)):
                                if clusterdata[j]==clusterdata[l]:
                                    tempvalue=freqmatrix[j][l]
                                    freqmatrix[j][l]=tempvalue+1/float(numit)#dividing by the number of iterations so the average will be between 0-1
                np.savetxt('../SEM/%s/FrequencyMatrix/Frequencymatrix_%s_rep=%s_basis=%s.txt'%(filetype,today,str(repl),str(basis)),freqmatrix)



        nmfmatrix=Freqaverage(repl,mutationlist,filetype,today)#averaging over all K
        np.savetxt('../SEM/%s/FrequencyMatrix/Frequencymatrix_%s_rep=%s_Average.txt'%(filetype,today,str(repl)),nmfmatrix)
        totalfreqmatrix=totalfreqmatrix+nmfmatrix/replicates#This line is a running average over all replicates. This is the final frequency matrix that is the culmination of this entire method

    out_df=pd.DataFrame(totalfreqmatrix, index=mutationlist,columns=mutationlist)
    outfile_name='../SEM/%s/TotalFrequencyMatrix_Average.txt'%(filetype)
    out_df.to_csv(outfile_name, header=True, index=False, sep='\t', mode='a')
    
    
if random_flag=='Yes':
    for repl in range(0,replicates):
        print(repl)
        ##randomly sampling mean and std in order to create new matrix
        error_df= pd.DataFrame(index=data_df.index.values, columns=data_df.loc[:,(assays_of_int,measure_mean_list)].columns)
        error_df.fillna(0,inplace=True)
        mean_df.apply(sample_from_SEM, axis=0)
        erroredmat=error_df.values
        
        erroredmat=randomizeclusters(erroredmat)
        
        np.savetxt('../SEM/RandomClustering/%s/Randommatrices/randommatrix_%s_%s.txt'%(filetype,today,str(repl)),erroredmat)
        
        #standize the matrix values using the max and min within each assay column (value-min)/(max-min) 
        #where max and min are column specific
        standmat=standardizemat(erroredmat)
        np.savetxt('../SEM/RandomClustering/%s/Standardizedmatrices/Standardmatrix_%s_%s.txt'%(filetype,today,str(repl)),standmat)
        freqmatrix=np.zeros((len(error_df),len(error_df)))

        ##############################################
        ###Begin clustering algorithm using the standardized sampled matrix you just created above
        nmfmatrix=np.zeros(shape=(len(error_df),len(error_df)))
        for basis in range(2,4):#Will perform nmf/kmeans method over each K between this range
            for clustk in range(2,4):
                if basis==clustk:#Only running if K=K for nnmf and Kmeans. This could be removed if you want all iterations 2,2; 2,3; 2,4;....8,7; 8,8
                    freqmatrix=np.zeros((len(error_df),len(error_df)))
                    for i in range(0,numit):
                        print('This is replicate',repl,'and iteration=',i,' with basis and clustk=',basis)

                        W,H=nmf(standmat,clustk)#Creating basis vectors. ONly W is used in this script, but H is used in the Assay clustering script
                        clusterdata=KMeans(n_clusters=clustk).fit_predict(W)

                        for j in range (0,len(clusterdata)):
                            #Quantifying if two compounds occur in the same cluster or not. Appending a an overall frequency matrix specific to this replicate.
                            #So for each replicate, a frequency matrix created which calculates how often all compounds cluster together over the number of numit for Each K
                            for l in range (0,len(clusterdata)):
                                if clusterdata[j]==clusterdata[l]:
                                    tempvalue=freqmatrix[j][l]
                                    freqmatrix[j][l]=tempvalue+1/float(numit)#dividing by the number of iterations so the average will be between 0-1
                np.savetxt('../SEM/RandomClustering/%s/FrequencyMatrix/Frequencymatrix_%s_rep=%s_basis=%s.txt'%(filetype,today,str(repl),str(basis)),freqmatrix)



        nmfmatrix=Freqaverage(repl,mutationlist,filetype,today)#averaging over all K
        totalfreqmatrix=totalfreqmatrix+nmfmatrix/replicates#This line is a running average over all replicates. This is the final frequency matrix that is the culmination of this entire method

    out_df=pd.DataFrame(totalfreqmatrix, index=mutationlist,columns=mutationlist)
    outfile_name='../SEM/RandomClustering/%s/TotalFrequencyMatrix_Average.txt'%(filetype)
    out_df.to_csv(outfile_name, header=True, index=False, sep='\t', mode='a')

0
This is replicate 0 and iteration= 0  with basis and clustk= 2
This is replicate 0 and iteration= 1  with basis and clustk= 2
This is replicate 0 and iteration= 2  with basis and clustk= 2
This is replicate 0 and iteration= 3  with basis and clustk= 2
This is replicate 0 and iteration= 4  with basis and clustk= 2
This is replicate 0 and iteration= 5  with basis and clustk= 2
This is replicate 0 and iteration= 6  with basis and clustk= 2
This is replicate 0 and iteration= 7  with basis and clustk= 2
This is replicate 0 and iteration= 8  with basis and clustk= 2
This is replicate 0 and iteration= 9  with basis and clustk= 2
This is replicate 0 and iteration= 0  with basis and clustk= 3
This is replicate 0 and iteration= 1  with basis and clustk= 3
This is replicate 0 and iteration= 2  with basis and clustk= 3
This is replicate 0 and iteration= 3  with basis and clustk= 3
This is replicate 0 and iteration= 4  with basis and clustk= 3
This is replicate 0 and iteration= 5  with basis and 

This is replicate 6 and iteration= 0  with basis and clustk= 3
This is replicate 6 and iteration= 1  with basis and clustk= 3
This is replicate 6 and iteration= 2  with basis and clustk= 3
This is replicate 6 and iteration= 3  with basis and clustk= 3
This is replicate 6 and iteration= 4  with basis and clustk= 3
This is replicate 6 and iteration= 5  with basis and clustk= 3
This is replicate 6 and iteration= 6  with basis and clustk= 3
This is replicate 6 and iteration= 7  with basis and clustk= 3
This is replicate 6 and iteration= 8  with basis and clustk= 3
This is replicate 6 and iteration= 9  with basis and clustk= 3
7
This is replicate 7 and iteration= 0  with basis and clustk= 2
This is replicate 7 and iteration= 1  with basis and clustk= 2
This is replicate 7 and iteration= 2  with basis and clustk= 2
This is replicate 7 and iteration= 3  with basis and clustk= 2
This is replicate 7 and iteration= 4  with basis and clustk= 2
This is replicate 7 and iteration= 5  with basis and 